# Utility Chains

##### Summarizing Documents

**load_summarize_chain**

In [1]:
# from langchain.llms import OpenAI # This import has been replaced by the below
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
llm = OpenAI(temperature=0.9)

In [5]:
# Reading the document

with open("D:/Lab Setup/Generic-and-Utility-Chains/sample.txt") as f:
    data = f.read()

**When it comes to document processing, breaking a large document into smaller, more manageable chunks is essential**

**Splitting text**

In [6]:
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)

**Create multiple documents**

In [7]:
docs = [Document(page_content=t) for t in texts]

In [8]:
docs

[Document(page_content="India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.\n\nIndia is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected government.\n\nIndia has a rich cultural heritage that has evolved over

**To create an instance of load_summarizer_chain, we need to provide three arguments.**

**Firstly, we need to pass the desired large language model that will be used to query the user input. Secondly, we specify the type of langchain chain to be used for summarizing documents.**

**Lastly, we can set the verbose argument to True if we want to see all the intermediate steps involved in processing the user request and generating the output.**


In [9]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.invoke(docs)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Write a concise summary of the following:


"India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.

India is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected government.

India has a ri

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Write a concise summary of the following:


"

India is a culturally diverse and vibrant country in South Asia, known for its rich history, geographical features, and religious diversity. It is the seventh-largest country by land area and second-most populous. India has a federal parliamentary democratic republic form of government with 28 states and 9 union territories. The country's cultural heritage is influenced by various religions, and festivals are celebrated harmoniously. Its history includes ancient civilizations, powerful empires, and a struggle for independence from British colonial rule. India's economy is one of the fastest-growing, with a focus on services, industry, and major cities as centers of business and innovation. However, the country also faces socio-economic challenges and prioritizes education to uplift marginalized communities. Indian professionals have made significant contributions globally, and the Indian film industry and cuisine h

{'input_documents': [Document(page_content="India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.\n\nIndia is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected government.\n\nIndia has a rich cultural heritage t

# HTTP Requests

##### LLMRequestsChain

In [10]:
from langchain.chains import LLMRequestsChain, LLMChain

In [11]:
template = """
Extract the answer to the question '{query}' or say "not found" if the information is not available.
{requests_result}
"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [12]:
llm=OpenAI()

In [14]:
chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=PROMPT))

**Preparing the question & inputs to the http request**

In [15]:
question = "What is the capital of india?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}

In [16]:
chain.invoke(inputs)

{'query': 'What is the capital of india?',
 'url': 'https://www.google.com/search?q=What+is+the+capital+of+india?',
 'output': '\nNew Delhi'}

**Let's look at the internal functioning**

In [17]:
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(  # type: ignore[attr-defined]
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

